In [1]:
import torch
import torchvision
import numpy as np
torch.__version__

'1.4.0'

In [ ]:
torchvision.__version__

### shape, size 一个是属性，一个是方法

In [ ]:
a = torch.tensor([1,2,3])
a.shape, a.size(), type(a.shape), type(a.size())

In [ ]:
a.shape[0], a.size()[0], a.numel()  #元素个数

In [ ]:
na = np.array([[1,2,3], [4,5,6]])
nb = torch.tensor(na)
nc = torch.as_tensor(na)
nd = torch.from_numpy(na)

In [ ]:
nb[:,1] = 55
na, nb

In [ ]:
nc[:,1] = 55
na, nc

In [ ]:
nd[:1] = 55
na, nd

In [ ]:
ne = [[1,2,3], [4,5,6]]
nf = torch.as_tensor(ne)
nf[:,1] = 55
ne, nf

### 可见，as-tensor, from-numpy,都是改变numpy自身，直接tensor基本等于复制进去
### 但是对list进行as-tensor仍然是复制

In [ ]:
a = torch.arange(1, 10)
a1 = a.view(3,3)
a.shape, a1.shape

### 步长 stride

数据是在内存里单向连续分布，所以所谓的数组，矩阵，不过是告诉内存，跨多少个元素算一行，跨多少个元素算一列
对元素的view进行改变，比如转置，改变的貌似只是stride，内存本身没变

In [ ]:
a.stride(), a1.stride()

以上是正常数组，每三个元素一行，每个元素间隔一个，为一列

以下是转置后，每隔三个元素一列（1,4,7），但每隔一个元素就是1行了(1,2,3)

In [ ]:
a2 = a1.permute(1, 0)
a2, a2.stride() 

可以通过`contiguous`把它重新捋顺

In [ ]:
if not a2.is_contiguous():
    a2 = a2.contiguous()
a2, a2.stride(), a2.is_contiguous()

很显然，`contiguous`不改变数组表现形式，只改变内存方式

### 把数组传进去当形状 xxxx_like

In [ ]:
value = torch.randn(3, 4)
zeros = torch.zeros_like(value)
ones  = torch.ones_like(value)
value, zeros, ones

In [ ]:
s = torch.full((2,3), fill_value=3, dtype=torch.int32)  # FILL
s

In [ ]:
a = torch.arange(24).reshape(4, 6)
b, c, d = a.chunk(3, dim=1)
e = torch.arange(25).reshape(5, 5)
f, g, h = e.chunk(3)
f, g, h

### 一些特殊索引（配合gather）

根据形状对应的列，取列里面的第几个元素
根据形状对应的行，取行里面的第几个元素

In [ ]:
ids    = [
    [0,1,3,0,3,2],
    [0,1,2,0,3,2],
    [0,1,1,0,3,2]
]
indexs = torch.tensor(ids)
torch.gather(a, dim=0, index=indexs) # 不能传入ids，必须是tensor

# 按列取时列要写满，行无所谓

In [ ]:
index2 = torch.tensor([
    [0,1,2],
    [5,5,5],
    [1,1,1],
    [2,3,4]
])
torch.gather(a, dim=1, index=index2)

# 按行取时行要写满，列无所谓

### 挤压 squeeze

把维度为1的通通压缩掉（即去掉只有一个子元素的的大括号
反向`unsqueeze`则是凭空加一个大括号, 加在哪一级由参数的`dim`决定

In [ ]:
em = torch.tensor([[[[[1]]]]])
em.squeeze()

In [ ]:
torch.tensor([1,2,3]).unsqueeze(1) # 这样就堆叠起来了

In [ ]:
m = torch.arange(24).reshape(2,3,4)
m.unsqueeze(0).shape, m.unsqueeze(1).shape, m.unsqueeze(2).shape

In [ ]:
m.permute(2,0,1)

### 特殊索引2 配合take

是把元素摊平按个数取的

In [ ]:
index_v = torch.tensor([
    [5, 6, 7, 12],
    [0, 1, 2, 3]
])
(a*2).take(index_v)

### 条件取值 (类三元表达式）

In [ ]:
torch_value1 = torch.arange(1, 10).view(3, 3)
torch_value2 = torch.full((3, 3), 88, dtype=torch.int64)

condition = torch.tensor([
    [True, False, True],
    [True, False, True],
    [True, False, True]
])

new_torch_value = torch.where(condition, torch_value1, torch_value2)  # if True torch_value1 else torch_value2
new_torch_value

In [ ]:
x = torch.randn(3, 3)
y = torch.zeros(3, 3)
condition = x > 0
z = torch.where(condition, x, y)
z

### Sigmoid

In [ ]:
torch.tensor([0], dtype=torch.int64).sigmoid()

In [ ]:
torch.zeros(2).sigmoid()

In [ ]:
torch_value = torch.arange(9).view(3, 3)
print(torch_value)
argmax_index = torch.argmax(torch_value, dim=1, keepdim=True)
print(argmax_index)

# 跟topk取top1差不多，但不能keepdim
print(torch_value.topk(1, dim=0)) 

### topk

默认按行取每行最大值(dim=1), 按列取时返回每列里的索引
同时返出value和index, 比`argmax`多一项返回

In [ ]:
print(a)
a.topk(3) # 如果有维度，则按维度取topk

In [ ]:
a.topk(2, dim=0)

### 特殊索引 boolean索引器